In [2]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB

In [15]:
#load dataset
dataset_amazon = pd.read_csv('./OVH_emotions/sentiment labelled sentences/amazon_cells_labelled.txt',\
                     sep="\t",
                      names=['text', 'label'])

dataset_yelp = pd.read_csv('./OVH_emotions/sentiment labelled sentences/yelp_labelled.txt',\
                     sep="\t",
                      names=['text', 'label'])

dataset_imdb = pd.read_csv('./OVH_emotions/sentiment labelled sentences/imdb_labelled.txt',\
                     sep="\t",
                      names=['text', 'label'])


full_dataset = pd.concat([dataset_amazon,dataset_yelp,dataset_imdb])

In [16]:
full_dataset.shape

(2748, 2)